In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
from sklearn.ensemble import GradientBoostingClassifier
import time
import math

start=time.time()
le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/bank_churn.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

#transform data into numeric value
dataset['Geography']=le.fit_transform(dataset['Geography'])
dataset['Gender']=le.fit_transform(dataset['Gender'])

#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['Exited','CustomerId','RowNumber','Surname'],axis=1)
y=dataset["Exited"]
y = le.fit_transform(y)
#imblearn SMOTE
#smote_enn = SMOTE()
smote_enn = SMOTEENN()
X, y = smote_enn.fit_resample(X, y)

#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)


end = time.time()
print("The time of execution of preprocess:",
      (end-start), "s")


The time of execution of preprocess: 0.8156623840332031 s


In [2]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-dt-cs-auc.xlsx', index=False)

Best: 0.814165 using {'class_weight': {0: 70, 1: 100}}
0.810944 (0.011254) with: {'class_weight': {0: 10, 1: 100}}
0.811068 (0.009032) with: {'class_weight': {0: 20, 1: 100}}
0.808208 (0.010481) with: {'class_weight': {0: 30, 1: 100}}
0.809512 (0.011269) with: {'class_weight': {0: 40, 1: 100}}
0.808829 (0.012300) with: {'class_weight': {0: 50, 1: 100}}
0.810843 (0.009172) with: {'class_weight': {0: 60, 1: 100}}
0.814165 (0.008571) with: {'class_weight': {0: 70, 1: 100}}
0.810611 (0.011942) with: {'class_weight': {0: 80, 1: 100}}
0.810348 (0.011033) with: {'class_weight': {0: 90, 1: 100}}
0.810688 (0.008821) with: {'class_weight': {0: 100, 1: 100}}


In [3]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-dt-cs-f1.xlsx', index=False)

Best: 0.812763 using {'class_weight': {0: 20, 1: 100}}
0.807859 (0.009943) with: {'class_weight': {0: 10, 1: 100}}
0.812763 (0.010258) with: {'class_weight': {0: 20, 1: 100}}
0.809400 (0.009122) with: {'class_weight': {0: 30, 1: 100}}
0.811721 (0.010233) with: {'class_weight': {0: 40, 1: 100}}
0.812098 (0.011663) with: {'class_weight': {0: 50, 1: 100}}
0.811029 (0.011782) with: {'class_weight': {0: 60, 1: 100}}
0.811612 (0.009143) with: {'class_weight': {0: 70, 1: 100}}
0.811505 (0.012180) with: {'class_weight': {0: 80, 1: 100}}
0.811595 (0.008151) with: {'class_weight': {0: 90, 1: 100}}
0.811864 (0.009899) with: {'class_weight': {0: 100, 1: 100}}


In [4]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-dt-cs-gmean.xlsx', index=False)

Best: 0.811017 using {'class_weight': {0: 60, 1: 100}}
0.810320 (0.008572) with: {'class_weight': {0: 10, 1: 100}}
0.809835 (0.009812) with: {'class_weight': {0: 20, 1: 100}}
0.806376 (0.009396) with: {'class_weight': {0: 30, 1: 100}}
0.809015 (0.011559) with: {'class_weight': {0: 40, 1: 100}}
0.810152 (0.009573) with: {'class_weight': {0: 50, 1: 100}}
0.811017 (0.010821) with: {'class_weight': {0: 60, 1: 100}}
0.809677 (0.009050) with: {'class_weight': {0: 70, 1: 100}}
0.808613 (0.012124) with: {'class_weight': {0: 80, 1: 100}}
0.809598 (0.008635) with: {'class_weight': {0: 90, 1: 100}}
0.809177 (0.010324) with: {'class_weight': {0: 100, 1: 100}}


In [5]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-lr-cs-auc.xlsx', index=False)

Best: 0.769339 using {'class_weight': {0: 10, 1: 100}}
0.769339 (0.012862) with: {'class_weight': {0: 10, 1: 100}}
0.764359 (0.013077) with: {'class_weight': {0: 20, 1: 100}}
0.762744 (0.012675) with: {'class_weight': {0: 30, 1: 100}}
0.757742 (0.013253) with: {'class_weight': {0: 40, 1: 100}}
0.755182 (0.013242) with: {'class_weight': {0: 50, 1: 100}}
0.733567 (0.043867) with: {'class_weight': {0: 60, 1: 100}}
0.745915 (0.029932) with: {'class_weight': {0: 70, 1: 100}}
0.742982 (0.027778) with: {'class_weight': {0: 80, 1: 100}}
0.741881 (0.021254) with: {'class_weight': {0: 90, 1: 100}}
0.727390 (0.042131) with: {'class_weight': {0: 100, 1: 100}}


In [6]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-lr-cs-f1.xlsx', index=False)

Best: 0.678305 using {'class_weight': {0: 90, 1: 100}}
0.377505 (0.004954) with: {'class_weight': {0: 10, 1: 100}}
0.451903 (0.011734) with: {'class_weight': {0: 20, 1: 100}}
0.524770 (0.022114) with: {'class_weight': {0: 30, 1: 100}}
0.570953 (0.013062) with: {'class_weight': {0: 40, 1: 100}}
0.614489 (0.013727) with: {'class_weight': {0: 50, 1: 100}}
0.632462 (0.013934) with: {'class_weight': {0: 60, 1: 100}}
0.656442 (0.017842) with: {'class_weight': {0: 70, 1: 100}}
0.667050 (0.019180) with: {'class_weight': {0: 80, 1: 100}}
0.678305 (0.017237) with: {'class_weight': {0: 90, 1: 100}}
0.676504 (0.030726) with: {'class_weight': {0: 100, 1: 100}}


In [7]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-lr-cs-gmean.xlsx', index=False)

Best: 0.662733 using {'class_weight': {0: 100, 1: 100}}
0.133363 (0.017964) with: {'class_weight': {0: 10, 1: 100}}
0.304942 (0.020957) with: {'class_weight': {0: 20, 1: 100}}
0.422849 (0.031812) with: {'class_weight': {0: 30, 1: 100}}
0.493230 (0.017795) with: {'class_weight': {0: 40, 1: 100}}
0.555108 (0.018246) with: {'class_weight': {0: 50, 1: 100}}
0.587652 (0.013102) with: {'class_weight': {0: 60, 1: 100}}
0.621611 (0.019141) with: {'class_weight': {0: 70, 1: 100}}
0.640475 (0.020205) with: {'class_weight': {0: 80, 1: 100}}
0.659759 (0.019062) with: {'class_weight': {0: 90, 1: 100}}
0.662733 (0.034820) with: {'class_weight': {0: 100, 1: 100}}


In [8]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-svm-cs-auc.xlsx', index=False)

Best: 0.697578 using {'class_weight': {0: 100, 1: 100}}
0.568127 (0.049944) with: {'class_weight': {0: 10, 1: 100}}
0.623749 (0.048633) with: {'class_weight': {0: 20, 1: 100}}
0.683948 (0.037779) with: {'class_weight': {0: 30, 1: 100}}
0.624568 (0.025024) with: {'class_weight': {0: 40, 1: 100}}
0.648044 (0.013826) with: {'class_weight': {0: 50, 1: 100}}
0.656916 (0.013162) with: {'class_weight': {0: 60, 1: 100}}
0.665913 (0.014509) with: {'class_weight': {0: 70, 1: 100}}
0.674955 (0.013669) with: {'class_weight': {0: 80, 1: 100}}
0.691095 (0.013647) with: {'class_weight': {0: 90, 1: 100}}
0.697578 (0.015267) with: {'class_weight': {0: 100, 1: 100}}


In [9]:
from sklearn.metrics import f1_score, make_scorer
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-svm-cs-f1.xlsx', index=False)

Best: 0.634209 using {'class_weight': {0: 100, 1: 100}}
0.358015 (0.000086) with: {'class_weight': {0: 10, 1: 100}}
0.358015 (0.000086) with: {'class_weight': {0: 20, 1: 100}}
0.369906 (0.024352) with: {'class_weight': {0: 30, 1: 100}}
0.498062 (0.010584) with: {'class_weight': {0: 40, 1: 100}}
0.574270 (0.017879) with: {'class_weight': {0: 50, 1: 100}}
0.615639 (0.016888) with: {'class_weight': {0: 60, 1: 100}}
0.624075 (0.014413) with: {'class_weight': {0: 70, 1: 100}}
0.628481 (0.015808) with: {'class_weight': {0: 80, 1: 100}}
0.633947 (0.014679) with: {'class_weight': {0: 90, 1: 100}}
0.634209 (0.014801) with: {'class_weight': {0: 100, 1: 100}}


In [10]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-svm-cs-gmean.xlsx', index=False)

Best: 0.607068 using {'class_weight': {0: 100, 1: 100}}
0.000000 (0.000000) with: {'class_weight': {0: 10, 1: 100}}
0.000000 (0.000000) with: {'class_weight': {0: 20, 1: 100}}
0.049947 (0.100544) with: {'class_weight': {0: 30, 1: 100}}
0.393007 (0.017047) with: {'class_weight': {0: 40, 1: 100}}
0.512380 (0.021539) with: {'class_weight': {0: 50, 1: 100}}
0.573499 (0.019662) with: {'class_weight': {0: 60, 1: 100}}
0.587533 (0.016012) with: {'class_weight': {0: 70, 1: 100}}
0.597764 (0.018388) with: {'class_weight': {0: 80, 1: 100}}
0.606753 (0.016132) with: {'class_weight': {0: 90, 1: 100}}
0.607068 (0.016261) with: {'class_weight': {0: 100, 1: 100}}


In [11]:
# KNN
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
# Define grid parameters
balance = ['cosine', 'euclidean', 'manhattan','minkowski','chebyshev']
k_values = [3, 5, 7, 9, 11,13,15,17,19,21,23,25,27,29,31]  # Example values for K
param_grid = dict(metric=balance, n_neighbors=k_values)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-knn-cs-bank-auc.xlsx', index=False)

Best: 0.970989 using {'metric': 'manhattan', 'n_neighbors': 3}
0.716628 (0.009497) with: {'metric': 'cosine', 'n_neighbors': 3}
0.723873 (0.011513) with: {'metric': 'cosine', 'n_neighbors': 5}
0.727128 (0.010815) with: {'metric': 'cosine', 'n_neighbors': 7}
0.727583 (0.010464) with: {'metric': 'cosine', 'n_neighbors': 9}
0.727642 (0.011756) with: {'metric': 'cosine', 'n_neighbors': 11}
0.725774 (0.013289) with: {'metric': 'cosine', 'n_neighbors': 13}
0.724386 (0.015239) with: {'metric': 'cosine', 'n_neighbors': 15}
0.723233 (0.016182) with: {'metric': 'cosine', 'n_neighbors': 17}
0.721668 (0.016016) with: {'metric': 'cosine', 'n_neighbors': 19}
0.720118 (0.016388) with: {'metric': 'cosine', 'n_neighbors': 21}
0.717837 (0.016681) with: {'metric': 'cosine', 'n_neighbors': 23}
0.715255 (0.015901) with: {'metric': 'cosine', 'n_neighbors': 25}
0.713655 (0.016107) with: {'metric': 'cosine', 'n_neighbors': 27}
0.713045 (0.016319) with: {'metric': 'cosine', 'n_neighbors': 29}
0.712203 (0.01599

In [12]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-knn-cs-bank-f1.xlsx', index=False)

Best: 0.929189 using {'metric': 'manhattan', 'n_neighbors': 3}
0.668364 (0.008506) with: {'metric': 'cosine', 'n_neighbors': 3}
0.664327 (0.011198) with: {'metric': 'cosine', 'n_neighbors': 5}
0.668311 (0.008229) with: {'metric': 'cosine', 'n_neighbors': 7}
0.667872 (0.006086) with: {'metric': 'cosine', 'n_neighbors': 9}
0.666845 (0.011885) with: {'metric': 'cosine', 'n_neighbors': 11}
0.664436 (0.011530) with: {'metric': 'cosine', 'n_neighbors': 13}
0.663504 (0.015090) with: {'metric': 'cosine', 'n_neighbors': 15}
0.663876 (0.016104) with: {'metric': 'cosine', 'n_neighbors': 17}
0.662504 (0.015837) with: {'metric': 'cosine', 'n_neighbors': 19}
0.662383 (0.014297) with: {'metric': 'cosine', 'n_neighbors': 21}
0.661200 (0.013527) with: {'metric': 'cosine', 'n_neighbors': 23}
0.658277 (0.012860) with: {'metric': 'cosine', 'n_neighbors': 25}
0.658908 (0.014218) with: {'metric': 'cosine', 'n_neighbors': 27}
0.658037 (0.012307) with: {'metric': 'cosine', 'n_neighbors': 29}
0.657701 (0.01314

In [13]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-knn-cs-bank-gmean.xlsx', index=False)

Best: 0.925250 using {'metric': 'manhattan', 'n_neighbors': 3}
0.659706 (0.009826) with: {'metric': 'cosine', 'n_neighbors': 3}
0.654587 (0.011914) with: {'metric': 'cosine', 'n_neighbors': 5}
0.657065 (0.009400) with: {'metric': 'cosine', 'n_neighbors': 7}
0.654861 (0.007668) with: {'metric': 'cosine', 'n_neighbors': 9}
0.653104 (0.013660) with: {'metric': 'cosine', 'n_neighbors': 11}
0.649860 (0.013076) with: {'metric': 'cosine', 'n_neighbors': 13}
0.648945 (0.016557) with: {'metric': 'cosine', 'n_neighbors': 15}
0.648810 (0.017599) with: {'metric': 'cosine', 'n_neighbors': 17}
0.647431 (0.017236) with: {'metric': 'cosine', 'n_neighbors': 19}
0.647161 (0.015729) with: {'metric': 'cosine', 'n_neighbors': 21}
0.645992 (0.014414) with: {'metric': 'cosine', 'n_neighbors': 23}
0.643096 (0.013712) with: {'metric': 'cosine', 'n_neighbors': 25}
0.643306 (0.015631) with: {'metric': 'cosine', 'n_neighbors': 27}
0.642392 (0.013531) with: {'metric': 'cosine', 'n_neighbors': 29}
0.641900 (0.01418